In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    input_index = np.array(range(0, length, 24))
    label_index = input_index + 24* (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), pred_day, labels

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters

{'watershed': 'han_auto'}

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 20)

In [12]:
run_num = range(len(folder))
real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

ddday = 31
mmmonth = 12

#length = 1

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    #df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
                               colum_idx[idx], interpolation=interpolation_option[idx],
                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/han/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['가평_2015.xlsx', '가평_2016.xlsx', '가평_2017.xlsx', '가평_2018.xlsx', '가평_2019.xlsx', '가평_2020.xlsx'], ['서상_2015.xlsx', '서상_2016.xlsx', '서상_2017.xlsx', '서상_2018.xlsx', '서상_2019.xlsx', '서상_2020.xlsx'], ['의암호_2015.xlsx', '의암호_2016.xlsx', '의암호_2017.xlsx', '의암호_2018.xlsx', '의암호_2019.xlsx', '의암호_2020.xlsx']]
data/han/자동/가평_2015.xlsx
data/han/자동/가평_2016.xlsx
data/han/자동/가평_2017.xlsx
data/han/자동/가평_2018.xlsx
data/han/자동/가평_2019.xlsx
data/han/자동/가평_2020.xlsx
time range in files :  2015-01-01 00:00  ~  2020-7-31 23:00
data/han/자동/서상_2015.xlsx
data/han/자동/서상_2016.xlsx
data/han/자동/서상_2017.xlsx
data/han/자동/서상_2018.xlsx
data/han/자동/서상_2019.xlsx
data/han/자동/서상_2020.xlsx
time range in files :  2015-01-01 00:00  ~  2020-7-31 23:00
data/han/자동/의암호_2015.xlsx
data/han/자동/의암호_2016.xlsx
data/han/자동/의암호_2017.xlsx
data/han/자동/의암호_2018.xlsx
data/han/자동/의암호_2019.xlsx
data/han/자동/의암호_2020.xlsx
time ran

1/1 [==============================] - 0s 20ms/step - gen_loss: 10.1724 - disc_loss: 0.3869 - rmse: 0.3775 - val_loss: 0.3678
Epoch 52/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 13.1625 - disc_loss: 0.3555 - rmse: 0.4068 - val_loss: 0.4888
Epoch 53/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.7530 - disc_loss: 0.3615 - rmse: 0.4119 - val_loss: 0.3527
Epoch 54/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.9056 - disc_loss: 0.3764 - rmse: 0.3314 - val_loss: 0.4333
Epoch 55/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.6247 - disc_loss: 0.3693 - rmse: 0.3614 - val_loss: 0.4164
Epoch 56/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.0761 - disc_loss: 0.3494 - rmse: 0.4072 - val_loss: 0.5186
Epoch 57/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.1550 - disc_loss: 0.3556 - rmse: 0.4183 - val_loss: 0.3523
Epoch 58/2000
1/1 [=================

1/1 [==============================] - 0s 20ms/step - gen_loss: 9.6997 - disc_loss: 0.2848 - rmse: 0.4026 - val_loss: 0.3643
Epoch 110/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.4051 - disc_loss: 0.2847 - rmse: 0.4370 - val_loss: 0.3424
Epoch 111/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.0966 - disc_loss: 0.2972 - rmse: 0.3969 - val_loss: 0.4047
Epoch 112/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.6550 - disc_loss: 0.3015 - rmse: 0.3450 - val_loss: 0.3416
Epoch 113/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.3860 - disc_loss: 0.2821 - rmse: 0.3550 - val_loss: 0.3946
Epoch 114/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.9927 - disc_loss: 0.2869 - rmse: 0.3467 - val_loss: 0.4257
Epoch 115/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.6791 - disc_loss: 0.3085 - rmse: 0.3474 - val_loss: 0.3906
Epoch 116/2000
1/1 [=============

1/1 [==============================] - 0s 20ms/step - gen_loss: 6.7694 - disc_loss: 0.2534 - rmse: 0.3436 - val_loss: 0.2982
Epoch 168/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 7.5735 - disc_loss: 0.2430 - rmse: 0.3357 - val_loss: 0.3230
Epoch 169/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 9.0751 - disc_loss: 0.2697 - rmse: 0.3814 - val_loss: 0.3421
Epoch 170/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.8423 - disc_loss: 0.2587 - rmse: 0.3454 - val_loss: 0.3859
Epoch 171/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 16.1992 - disc_loss: 0.2740 - rmse: 0.5441 - val_loss: 0.3958
Epoch 172/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.5545 - disc_loss: 0.2586 - rmse: 0.3106 - val_loss: 0.4356
Epoch 173/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.2045 - disc_loss: 0.2658 - rmse: 0.3581 - val_loss: 0.3541
Epoch 174/2000
1/1 [==============

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9510 - disc_loss: 0.2424 - rmse: 0.2961 - val_loss: 0.3803
Epoch 226/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.7969 - disc_loss: 0.2394 - rmse: 0.3073 - val_loss: 0.3446
Epoch 227/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.6446 - disc_loss: 0.2528 - rmse: 0.3560 - val_loss: 0.3415
Epoch 228/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.6093 - disc_loss: 0.2507 - rmse: 0.3431 - val_loss: 0.3340
Epoch 229/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.8797 - disc_loss: 0.2432 - rmse: 0.2955 - val_loss: 0.3400
Epoch 230/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.0394 - disc_loss: 0.2501 - rmse: 0.3011 - val_loss: 0.3225
Epoch 231/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.0179 - disc_loss: 0.2486 - rmse: 0.2982 - val_loss: 0.4171
Epoch 232/2000
1/1 [===============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3360 - disc_loss: 0.1796 - rmse: 0.2974 - val_loss: 0.2885
Epoch 632/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.6512 - disc_loss: 0.1906 - rmse: 0.3057 - val_loss: 0.2640
Epoch 633/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 13.9248 - disc_loss: 0.1982 - rmse: 0.2858 - val_loss: 0.2641
Epoch 634/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 6.1932 - disc_loss: 0.1974 - rmse: 0.3286 - val_loss: 0.3181
Epoch 635/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7843 - disc_loss: 0.1863 - rmse: 0.2785 - val_loss: 0.3231
Epoch 636/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.1001 - disc_loss: 0.1924 - rmse: 0.3052 - val_loss: 0.3321
Epoch 637/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.4014 - disc_loss: 0.1929 - rmse: 0.3138 - val_loss: 0.3076
Epoch 638/2000
1/1 [==============

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.6873 - disc_loss: 0.1832 - rmse: 0.3202 - val_loss: 0.3218
Epoch 690/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.2355 - disc_loss: 0.1736 - rmse: 0.3078 - val_loss: 0.3150
Epoch 691/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.1876 - disc_loss: 0.2012 - rmse: 0.3007 - val_loss: 0.2838
Epoch 692/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4181 - disc_loss: 0.1836 - rmse: 0.2686 - val_loss: 0.3074
Epoch 693/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9024 - disc_loss: 0.1822 - rmse: 0.3045 - val_loss: 0.3223
Epoch 694/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.6516 - disc_loss: 0.1976 - rmse: 0.2933 - val_loss: 0.2899
Epoch 695/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.8245 - disc_loss: 0.1906 - rmse: 0.2926 - val_loss: 0.3418
Epoch 696/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 6.2890 - disc_loss: 0.1799 - rmse: 0.3289 - val_loss: 0.3050
Epoch 748/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3421 - disc_loss: 0.1843 - rmse: 0.2855 - val_loss: 0.2764
Epoch 749/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7750 - disc_loss: 0.1764 - rmse: 0.2629 - val_loss: 0.2622
Epoch 750/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.2488 - disc_loss: 0.1798 - rmse: 0.2751 - val_loss: 0.3045
Epoch 751/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.2476 - disc_loss: 0.1845 - rmse: 0.2720 - val_loss: 0.2968
Epoch 752/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.5657 - disc_loss: 0.1829 - rmse: 0.3188 - val_loss: 0.2563
Epoch 753/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.6449 - disc_loss: 0.1878 - rmse: 0.2744 - val_loss: 0.3047
Epoch 754/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 5.5208 - disc_loss: 0.1781 - rmse: 0.3077 - val_loss: 0.3072
Epoch 806/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7451 - disc_loss: 0.1771 - rmse: 0.2839 - val_loss: 0.2463
Epoch 807/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.9012 - disc_loss: 0.1888 - rmse: 0.2938 - val_loss: 0.3044
Epoch 808/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.7062 - disc_loss: 0.1855 - rmse: 0.3306 - val_loss: 0.3009
Epoch 809/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7650 - disc_loss: 0.1872 - rmse: 0.2400 - val_loss: 0.2518
Epoch 810/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1044 - disc_loss: 0.1796 - rmse: 0.2854 - val_loss: 0.3002
Epoch 811/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9054 - disc_loss: 0.1880 - rmse: 0.3202 - val_loss: 0.3463
Epoch 812/2000
1/1 [===============

1/1 [==============================] - 0s 20ms/step - gen_loss: 14.0442 - disc_loss: 0.1678 - rmse: 0.2572 - val_loss: 0.3564
Epoch 864/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1615 - disc_loss: 0.1679 - rmse: 0.2773 - val_loss: 0.2634
Epoch 865/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 4.0696 - disc_loss: 0.1806 - rmse: 0.2486 - val_loss: 0.3175
Epoch 866/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4863 - disc_loss: 0.1741 - rmse: 0.2818 - val_loss: 0.2824
Epoch 867/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.2332 - disc_loss: 0.1740 - rmse: 0.3183 - val_loss: 0.4053
Epoch 868/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.4560 - disc_loss: 0.1830 - rmse: 0.3183 - val_loss: 0.2883
Epoch 869/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 13.7853 - disc_loss: 0.1808 - rmse: 0.2793 - val_loss: 0.2913
Epoch 870/2000
1/1 [=============

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  6
out_num_features :  1


In [52]:
print(real_df_all.shape)
rnn_target_column = 'chl-a'
print(rnn_target_column)

#df_length = real_df_all.shape[0] - 3000
df_length = real_df_all.shape[0]

fake_df = real_df_all.iloc[:df_length,:]

(48936, 36)
chl-a


In [53]:
#train_df, val_df, test_df, test_df2 = dataset_slice(fake_df, 0.8, 0.1, 0.1)
#teg_check = 'train=8, val=1, test=1'

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

#val_df, test_df,train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'val=1, test=1, train=8'

#test_df, val_df, train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'test=1, val=1, train=8'

#val_df, train_df, test_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'val=1, train=8, test=1'

#test_df, train_df, val_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'test=1, train=8, val=1'

train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, test=1, val=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (34255, 36) val_df.shape :  (9788, 36) test_df.shape: (4893, 36)


In [54]:
#fake_df.shape[0]/8760

In [55]:
plt.figure()
plt.plot(real_df_all.iloc[:, out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [56]:
plt.figure()
plt.plot(val_df.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [58]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

target_col_idx :  7
out_num_features :  1


In [59]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}

In [60]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

model_path = "save/" + watershed + "models/" + pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    if not os.path.exists('save/' + watershed):
        os.makedirs('save/' + watershed)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/han/models/chlorophyll-a/


In [61]:
rnn_epochs

20

## 모델 학습

In [ ]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=50,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")

Epoch 1/50
10/10 [==============================] - ETA: 0s - loss: 0.5872 - mean_absolute_error: 0.4961 - nse: 0.2125
Epoch 00001: val_loss improved from inf to 0.73992, saving model to save/best_model.h5
10/10 [==============================] - 8s 776ms/step - loss: 0.5872 - mean_absolute_error: 0.4961 - nse: 0.2125 - val_loss: 0.7399 - val_mean_absolute_error: 0.6520 - val_nse: 0.3011
Epoch 2/50
10/10 [==============================] - ETA: 0s - loss: 0.3500 - mean_absolute_error: 0.3801 - nse: 0.5108
Epoch 00002: val_loss improved from 0.73992 to 0.53518, saving model to save/best_model.h5
10/10 [==============================] - 8s 757ms/step - loss: 0.3500 - mean_absolute_error: 0.3801 - nse: 0.5108 - val_loss: 0.5352 - val_mean_absolute_error: 0.5695 - val_nse: 0.4930
Epoch 3/50
10/10 [==============================] - ETA: 0s - loss: 0.2695 - mean_absolute_error: 0.3343 - nse: 0.6439
Epoch 00003: val_loss improved from 0.53518 to 0.32210, saving model to save/best_model.h5
10/1

Epoch 22/50
10/10 [==============================] - ETA: 0s - loss: 0.1952 - mean_absolute_error: 0.2815 - nse: 0.7451
Epoch 00022: val_loss did not improve from 0.24684
10/10 [==============================] - 7s 745ms/step - loss: 0.1952 - mean_absolute_error: 0.2815 - nse: 0.7451 - val_loss: 0.2886 - val_mean_absolute_error: 0.3738 - val_nse: 0.7259
Epoch 23/50
10/10 [==============================] - ETA: 0s - loss: 0.2198 - mean_absolute_error: 0.3008 - nse: 0.7257
Epoch 00023: val_loss did not improve from 0.24684
10/10 [==============================] - 7s 746ms/step - loss: 0.2198 - mean_absolute_error: 0.3008 - nse: 0.7257 - val_loss: 0.4406 - val_mean_absolute_error: 0.4440 - val_nse: 0.5819
Epoch 24/50
10/10 [==============================] - ETA: 0s - loss: 0.1898 - mean_absolute_error: 0.2824 - nse: 0.7376
Epoch 00024: val_loss did not improve from 0.24684
10/10 [==============================] - 7s 747ms/step - loss: 0.1898 - mean_absolute_error: 0.2824 - nse: 0.7376 - v

10/10 [==============================] - ETA: 0s - loss: 0.1240 - mean_absolute_error: 0.2355 - nse: 0.8072
Epoch 00045: val_loss did not improve from 0.24684
10/10 [==============================] - 8s 754ms/step - loss: 0.1240 - mean_absolute_error: 0.2355 - nse: 0.8072 - val_loss: 0.4305 - val_mean_absolute_error: 0.4324 - val_nse: 0.5948
Epoch 46/50
10/10 [==============================] - ETA: 0s - loss: 0.1494 - mean_absolute_error: 0.2373 - nse: 0.7969
Epoch 00046: val_loss did not improve from 0.24684
10/10 [==============================] - 7s 744ms/step - loss: 0.1494 - mean_absolute_error: 0.2373 - nse: 0.7969 - val_loss: 0.4767 - val_mean_absolute_error: 0.4508 - val_nse: 0.5520
Epoch 47/50
 1/10 [==>...........................] - ETA: 0s - loss: 0.1234 - mean_absolute_error: 0.2317 - nse: 0.8572

## core / window.py / 

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [28]:
#train_df.shape, val_df.shape, test_df.shape

In [29]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvbnfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************            
#                   SSSSSSSS               ******            ******              *****    *****                      
#        SSSS         S  SSS               ******            ******              *****    *****                     
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [ ]:
from core.models import GRUModel, compile

In [ ]:
model112 = GRUModel(rnn_out_steps, out_num_features)
model112.load_weights(model_path+"gru.ckpt")
compile(model112)


In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    #model=model112,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    model=model112,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
##.evaluate(multi_window.train)
#model112.evaluate(multi_window.val.repeat(-1), steps=100)

In [ ]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    #print(df.shape)
    #print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    

    predictions = model(inputs).numpy()
    
    print("input : ", inputs.shape, "labels : ", labels.shape, "predictions : ", predictions.shape)
    #print(predictions.shape)
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    print("                label_day.shape : ",label_day.shape, "pred_day.shape : ", pred_day.shape)
    
    #plt.figure()
    #input_index = np.array(range(0, length, 24))
    #label_index = input_index + 24* (7 + predict_day)
    #plt.plot(input_index, inputs_day[:, 0, :], label='input')
    #plt.plot(label_index, label_day[:, predict_day, :], label='label')
    #plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    #plt.legend()
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), inputs_day, label_day, pred_day
    

In [ ]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()
